In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [3]:
data=pd.read_csv("Churn_Modelling.csv")
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
label_encoder_gender=LabelEncoder()
data["Gender"]=label_encoder_gender.fit_transform(data["Gender"])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [8]:
from sklearn.preprocessing import OneHotEncoder
onehot=OneHotEncoder()
geo_encoder=onehot.fit_transform(data[["Geography"]])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [9]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
data=pd.concat([data.drop("Geography",axis=1), geo_encoded_df],axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [11]:
with open ("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)
with open ("onehot.pkl","wb") as file:
    pickle.dump(onehot,file)

In [12]:
X=data.drop("Exited",axis=1)
y=data["Exited"]
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.25, random_state=32)

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler,file)

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [19]:
model=Sequential([
    Dense(64,activation="relu",input_shape=(x_train.shape[1],)),
    Dense(32,activation="relu"),
    Dense(1,activation="sigmoid")
])

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [36]:
lg_dir="logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
lg_dir

'logs/fit/20250123-105705'

In [37]:
tensorflow_callback=TensorBoard(log_dir=lg_dir,histogram_freq=1)

In [38]:
tensorflow_callback
early_stopping_callback=EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

In [39]:
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
235/235 [==============================] - 1s 5ms/step - loss: 0.3165 - accuracy: 0.8704 - val_loss: 0.3491 - val_accuracy: 0.8512
Epoch 2/100
235/235 [==============================] - 1s 5ms/step - loss: 0.3137 - accuracy: 0.8713 - val_loss: 0.3506 - val_accuracy: 0.8496
Epoch 3/100
235/235 [==============================] - 2s 7ms/step - loss: 0.3131 - accuracy: 0.8700 - val_loss: 0.3555 - val_accuracy: 0.8492
Epoch 4/100
235/235 [==============================] - 3s 11ms/step - loss: 0.3111 - accuracy: 0.8721 - val_loss: 0.3516 - val_accuracy: 0.8468
Epoch 5/100
235/235 [==============================] - 2s 8ms/step - loss: 0.3108 - accuracy: 0.8715 - val_loss: 0.3516 - val_accuracy: 0.8504
Epoch 6/100
235/235 [==============================] - 2s 7ms/step - loss: 0.3089 - accuracy: 0.8736 - val_loss: 0.3535 - val_accuracy: 0.8484


In [41]:
model.save("model.h5")

In [42]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [44]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 14956), started 0:03:06 ago. (Use '!kill 14956' to kill it.)

In [46]:
from tensorflow.keras.models import load_model
model=load_model("model.h5")
with open("label_encoder_gender.pkl","rb") as file:
    label_encoder_gen=pickle.load(file)
with open("onehot.pkl","rb") as file:
    one_hot_geo=pickle.load(file)
with open("scaler.pkl","rb") as file:
    scaler=pickle.load(file)

In [55]:
input_data={
    "CreditScore":600,
    "Geography":"France",
    "Gender":"Male",
    "Age":40,
    "Tenure":3,
    "Balance":60000,
    "NumOfProducts":2,
    "HasCrCard":1,
    "IsActiveMember":1,
    "EstimatedSalary":50000
}

In [61]:
geo_encoded=one_hot_geo.transform([[input_data["Geography"]]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot.get_feature_names_out(["Geography"]))
geo_encoded_df


c:\Users\htc\OneDrive\Desktop\Ann_class\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [62]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [63]:
input_df["Gender"]=label_encoder_gender.transform(input_df["Gender"])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [64]:
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [65]:
input_df=scaler.transform(input_df)
input_df

array([[-0.52557125,  0.91162201,  0.09954694, -0.6877138 , -0.26583204,
         0.7981571 ,  0.64489983,  0.96927188, -0.86633566,  1.00427581,
        -0.58104535, -0.57693971]])

In [67]:
prediction=model.predict(input_df)

1/1 [==============================] - 0s 34ms/step


In [68]:
prediction

array([[0.03574443]], dtype=float32)

In [69]:
prediction[0][0]

0.03574443

In [70]:
if prediction[0][0]>=0.5:
    print("The Person will chunk")
else:
    print("The person will not chunk")

The person will not chunk
